## Подключение всех необходимых библиотек

In [27]:
import os

import librosa
import numpy as np
import pandas as pd
import tqdm
from torch import nn
from torch.functional import F
from sklearn.model_selection import train_test_split

# Hyperparameters

In [28]:
N_MFCCS = 25

# Обработка аудио файлов и создание csv файла (датасета) с признаками

In [29]:
header = 'chroma_stft rms spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(N_MFCCS):
    header += f' mfcc{i}'
labeled_dataset = pd.read_csv("train_gt (1).csv")

## 1. Заполнение csv файла данными

In [49]:
if not os.path.exists("train_dataset.csv") or \
    pd.read_csv("train_dataset.csv").shape != N_MFCCS:
    for folder in ["train", "test"]:
        columns = (header if folder == "test" else header + " label").split()
        features = pd.DataFrame(columns=columns)
        for filename in tqdm.tqdm(os.listdir(folder), desc=folder):
            songname = os.path.join(folder, filename)
            y, sr = librosa.load(songname, mono=True)
            rms = librosa.feature.rms(y=y)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCCS)
            
            feature_row_data = [np.mean(chroma_stft), np.mean(rms), np.mean(spec_cent), np.mean(spec_bw), np.mean(rolloff), np.mean(zcr)]
            for e in mfcc:
                feature_row_data.append(np.mean(e))
            if folder == "train":
                feature_row_data.append(bool(labeled_dataset[labeled_dataset["Filename"] == filename]["Label"].values[0]))
            features.loc[filename] = pd.Series(feature_row_data, index=columns)
        features.to_csv(f"{folder}_dataset.csv")

test: 100%|██████████| 2870/2870 [02:20<00:00, 20.46it/s]


## Обработка датасета и разделение на тренировочную и тестовую выборки

In [50]:
df = pd.read_csv("train_dataset.csv", index_col=0)
X = df.drop(columns=["label"])
y = df["label"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
import torch.utils
import torch.utils.data


class MyDataset(torch.utils.data.Dataset):
        
    def __init__(self, features, labels, device=torch.device("cpu")):
        self.features = torch.tensor(features.values, dtype = torch.float32).to(device)
        self.labels = torch.tensor(labels.values, dtype = torch.float32).to(device)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, ind):
        return self.features[ind], self.labels[ind]

In [52]:
import torch.utils
import torch.utils.data

device = torch.device("mps") 
train_dataset = MyDataset(x_train, y_train, device)
test_dataset = MyDataset(x_test, y_test, device)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

## Архитектура нашей нейронной сети и ее обучение

In [53]:
class RecognizeNet(nn.Module):
    def __init__(self):
        super(RecognizeNet, self).__init__()
        self.layer1 = nn.Linear(6 + N_MFCCS, 100)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(100, 200)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(200, 100)
        self.act3 = nn.ReLU()
        self.layer4 = nn.Linear(100, 50)
        self.act4 = nn.ReLU()
        self.layer5 = nn.Linear(50, 2)
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.act1(x)
        x = self.layer2(x)
        x = self.act2(x)
        x = self.layer3(x)
        x = self.act3(x)
        x = self.layer4(x)
        x = self.act4(x)
        x = self.layer5(x)
        return x

In [54]:
model = RecognizeNet()
model.to(device)
total_epochs = 0

In [55]:
total_step = len(train_loader)
epochs = 1000
lr = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr)
loss_fn = nn.CrossEntropyLoss()

loss_list = []
acc_list = []
pbar = tqdm.tqdm(range(epochs))
for epoch in pbar:
    for i, batch in enumerate(train_loader):

        x, y = batch
        preds = model(x)

        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        preds = F.softmax(preds, dim=1)

        total = y.size(0)
        _, predicted = torch.max(preds.data, 1)
        true = y
        correct = (predicted == true).sum().item()
        acc_list.append(correct / total)
    total_epochs += 1
    if (epoch + 1) % 20 == 0:
        pbar.set_description('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(total_epochs, (total_epochs // epochs + 1) * epochs, loss.item(),
                        (correct / total) * 100))

  2%|▏         | 20/1000 [00:06<04:50,  3.37it/s]

Epoch [20/1000], Loss: 0.6585, Accuracy: 62.69%


  4%|▍         | 40/1000 [00:11<04:10,  3.84it/s]

Epoch [40/1000], Loss: 0.6674, Accuracy: 59.58%


  6%|▌         | 60/1000 [00:17<04:04,  3.84it/s]

...

Epoch [960/1000], Loss: 0.5386, Accuracy: 72.38%


 98%|█████████▊| 980/1000 [04:36<00:05,  3.37it/s]

Epoch [980/1000], Loss: 0.5753, Accuracy: 67.26%


100%|██████████| 1000/1000 [04:42<00:00,  3.54it/s]

Epoch [1000/2000], Loss: 0.5727, Accuracy: 69.04%


## Тестирование нашей нейронки

In [58]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        y_pred = model(x)
        _, predicted = torch.max(y_pred.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    print('Test Accuracy of the model on the test data: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the test data: 63.25951164111301 %


# Submission

In [59]:
data_scoring = pd.read_csv("test_dataset.csv", index_col=0)
sample_submission = pd.read_csv("test (1).csv")
model.eval()
with torch.no_grad():
    _, results = torch.max(model(torch.FloatTensor(data_scoring.values).to(device)).data, 1)

submission = pd.DataFrame({"Filename": data_scoring.index, "Label": results.to("cpu")})
submission = submission.sort_values(by="Filename")
sample_submission = sample_submission.sort_values(by="Filename")
sample_submission["Label"] = submission["Label"]
sample_submission = sample_submission.sort_index()
sample_submission.to_csv("test.csv", index=False, header=False) 